In [1]:
from microtc.utils import load_model, save_model
from sklearn.cluster import MiniBatchKMeans
import fasttext
import numpy as np
from microtc.utils import tweet_iterator
import pandas as pd
import re
from nltk.corpus import stopwords
from microtc.textmodel import TextModel
from sklearn import preprocessing
from sklearn.svm import  LinearSVC
from sklearn import metrics
import sys

pd.set_option('display.max_colwidth', None)

In [2]:
# Se obtiene el vector de cada palabra y es lo que se mete al Clustering
# es decir, es un Cluster de Vectores
kmeans_mini: MiniBatchKMeans = load_model('../kmeans/kmeans_mini_10000.obj')

C:\Users\Calderas\Anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator MiniBatchKMeans from version 1.0 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [3]:
# Necesitamos el fasttext para generar el Vector de la palabra a predecir
ft_mx = fasttext.load_model('../ft_data/MX.bin')
# el cluster almacena indices; requerimos Words para extrear la palabra
words = np.array(ft_mx.get_words())

In [30]:
fn = '../kmeans/ds/meoffendes2021_mx_Es_train.json'
#fn = '../kmeans/ds/haha2018_Es_train.json'
#fn = '../kmeans/ds/misoginia_Es_train.json'
train_df = pd.read_json(fn, lines=True)
test_df = pd.read_json(fn.replace("_train", "_test"), lines=True)

# recortarlos para prueba
train_df = train_df.loc[100:114, ['text', 'klass']]
test_df = test_df.loc[100:114, ['text', 'klass']]

X_train = train_df[['text']]
y_train = train_df[['klass']]

X_test = test_df[['text']] 
y_test = test_df[['klass']]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((15, 1), (15, 1), (15, 1), (15, 1))

In [31]:
# X_train original 
X_train.head(15)

,text
100,Inhala y exhala Mariana 😤 aún no puede saber que estás loca.
101,"""Quieres saber por que quiero acabar con esta loca?""#MariaIsabel #DivinaHelena #TeAmamosTanto @USUARIO"
102,"Otra vez,cuando irá a ser la noche que me duerma temprano la puta madre"
103,Ya valió verga se va a volver a inundar la cuidad horrible.
104,#YuyaFraudeGiveaway no mames las creaturitas hija de la verga😂😂😂😂😂😂😂😂😂
105,"Básicamente la ""Felicidad"" consiste en que te valga madre la vida de todos los demás y vivas la tuya como quieras."
106,Luego me pregunto porqué me va de la verga en el cole.
107,#DalasitoMentirosito Pudrete hijo de mil putas por todo lo que isiste
108,"Les dejo una pequeña reflexión, si les gusta me ayudan a compartirla?"
109,Ver a Normani en lugares gay me dan más vida


In [32]:
keep_idxs_cluster = {} # mantiene los indices de los cluster ya usados 
tm = TextModel()
size = X_train.size
for idx_train, row in X_train.iterrows():
    original_tweet = row['text']
    tokens = [t for t in tm.tokenize(original_tweet)  if t not in stopwords.words(['english', 'spanish'])]
    generated_tweet ="" # el nuevo tweet generado a partir de los tokens del original
    for tok in tokens:
        # extraer el vector del FastText y normalizarlo
        v_to_p = preprocessing.normalize([ft_mx.get_word_vector(tok)], norm='l1')
        # Prediccion de a cual Cluster pertenece el Vector del token
        idx_cluster_of_tok = kmeans_mini.predict(v_to_p)
        # extraer los indices de los vectores pertenecientes al Cluster del vector 
        idxs_into_cluster = np.where(kmeans_mini.labels_ == idx_cluster_of_tok)[0]
        # probar si ese indice de cluster ya fue usado y, en su caso, reutilizaro; de lo contrario generarlo 
        longest_w = keep_idxs_cluster.get(idx_cluster_of_tok[0])
        if longest_w is None:
            # extraemos la palabra mas larga dados los indices del cluster
            longest_w = max(words[idxs_into_cluster], key=len)
            # almacenamos la palabra mas larga como la llave/clave de todo ese cluster
            # y poder re-utilizarla en cada token que cae en ese cluster
            keep_idxs_cluster[idx_cluster_of_tok[0]] = longest_w 
        # agregamos el longest_w al tweet q se esta generando a raiz de los tokens del tweet original   
        generated_tweet = generated_tweet + " " + longest_w     
        
    X_train.at[idx_train,'text'] = [ original_tweet , generated_tweet]
    sys.stdout.write("\r%d/%d" % (idx_train, size))
    sys.stdout.flush()
    #print(X_train.at[idx_train,'text'])   

# guardar el X_train para re-utilizarlo
save_model(X_train, 'meoffendes2021_mx_Es_train-cluster.ds')

# guardar una copia pero con texto puro en vez de list
_ = X_train.copy()
def together(txt):
    return " ".join(txt)
_['text'] = _['text'].apply(together)
save_model(_, 'meoffendes2021_mx_Es_train-cluster-text.ds')


114/15

In [33]:
# X_train modificado
X_train.head(15)

,text
100,"[Inhala y exhala Mariana 😤 aún no puede saber que estás loca., respiraresvivir respiraresvivir yolandamaricarmen _descorazonamiento todaviano puedepasar sabemosque loquillaso]"
101,"[ ""Quieres saber por que quiero acabar con esta loca?""#MariaIsabel #DivinaHelena #TeAmamosTanto @USUARIO, sinquieres sabemosque noquieronoquieronoquiero terminarian loquillaso carlotaycamilaeljacal queridaamigaximenitanavarrete retoacaeliahecharseunacubetaconaguahelada |o|r|g|u|l|l|o|a|z|u|l|y|o|r|o|]"
102,"[Otra vez,cuando irá a ser la noche que me duerma temprano la puta madre, traaveezz enternecera creerselas trasnochesmx placidamente tempranititito reputisisisisisisima desmaadre]"
103,"[Ya valió verga se va a volver a inundar la cuidad horrible., valiendomadre baliendoberga enternecera volverselos inundaciones chilangocentrica horriblemente]"
104,"[#YuyaFraudeGiveaway no mames las creaturitas hija de la verga😂😂😂😂😂😂😂😂😂, fraudeelectoral nomamesnomamesnomames criaturillas progenitor _campaign=botoneramobile]"
105,"[Básicamente la ""Felicidad"" consiste en que te valga madre la vida de todos los demás y vivas la tuya como quieras., criminalogisticamente alegrandome consistoriales valiendomadre desmaadre vidayouthpdv criticarlos vivienderas sinquieres]"
106,"[Luego me pregunto porqué me va de la verga en el cole., luegoluego preguntandole enternecera baliendoberga antetokounmpo]"
107,"[#DalasitoMentirosito Pudrete hijo de mil putas por todo lo que isiste , mentirosamente susubordinacion progenitor chorrocientosmil chingadaderas almismotiempo]"
108,"[Les dejo una pequeña reflexión, si les gusta me ayudan a compartirla?, dejemoselo pequenisimos reflexionaremos alcontrario gustoculposo quimiayudas manzurtecomparte]"
109,"[Ver a Normani en lugares gay me dan más vida, viendolaa purposetour pinchesviejosridiculos homosexualizadores desentienden reciprocamente vidayouthpdv]"


In [34]:

X_train, X_test, y_train, y_test = X_train.squeeze(), X_test.squeeze(), y_train.squeeze(), y_test.squeeze()   

# probando con varias combinaciones del token_lst
tok_lst = [[-1], [-3], [3], [5], [-1, [2,1], [3,1], [2,2], [3,2], 2,3,4], [-1, [2,1], 3]]

lsvc = LinearSVC(penalty='l2', C=0.5, tol=1e-7)
res = []
for tok_l in tok_lst:
    tm = TextModel(token_list=tok_l).fit(X_train)
    m = lsvc.fit(tm.transform(X_train), y_train)
    p = m.predict(tm.transform(X_test))
    res.append([tok_l, metrics.f1_score(y_test, p, average="macro"), metrics.recall_score(y_test, p, average="macro")])
res_df = pd.DataFrame(res, columns=['tok_lst', 'f1_score', 'recall'])    
res_df

,tok_lst,f1_score,recall
0,[-1],0.423077,0.5
1,[-3],0.423077,0.5
2,[3],0.423077,0.5
3,[5],0.423077,0.5
4,"[-1, [2, 1], [3, 1], [2, 2], [3, 2], 2, 3, 4]",0.423077,0.5
5,"[-1, [2, 1], 3]",0.423077,0.5
